In [1]:
import pandas as pd

In [2]:
import json

In [3]:
from preprocess.preprocess import preprocess_text

In [4]:
from tqdm import tqdm_notebook

In [5]:
translations = json.load(open("./result-toxic.json"))

In [6]:
perspective_df = pd.read_csv("./train.csv")

In [7]:
toxic_classes = ['threat', 'insult', 'identity_hate', 'toxic']

In [8]:
perspective_df["real_toxic"] = (perspective_df[toxic_classes].sum(axis=1) > 0).astype(int)

In [9]:
translations

{'3e35ac57d49e9340': 'ты живешь в Японии, я знаю, ты говоришь по-японски.',
 'c098443299f29c42': 'все эти специальные взгляды могут быть разработаны в статье, они не нуждаются в неправильной известности в он ведет',
 'd5f8ca89283beeca': 'вы уверены, что это выглядело так, как будто они все пели, может быть, это Гари Робби Марк Говард и Джейсон в то же время видео кто-нибудь знает, когда видео будет показано, как правило, музыкальные видео показаны до выхода сингла, но это всего лишь дни',
 'f4413aedb7595cf9': 'кстати, нет таких китайских букв, как вы сказали',
 '35a39baa2b3f21ea': 'влияние нельзя забыть Бонни и Клайд фильм когда Уоррен Битти снимается в глаза эта последовательность происходит из фильма, а также в Брайан депалма с Капоне последовательность с багги спускаясь по ступенькам возникают из этого фильма Вуди Аллен относится к этому фильму, а также увидеть любовь и смерть и взять деньги и бежать',
 'a73da2d769cb2b52': 'API Error: The text size exceeds the maximum',
 '96605cb9ba

In [10]:
perspective_df["translate"] = perspective_df["id"].apply(lambda x: translations.get(x))

In [11]:
perspective_df["translate"] = perspective_df["translate"].fillna("API Error")

In [12]:
perspective_df.drop(perspective_df[perspective_df["translate"].str.startswith("API Error")].index, inplace=True)

In [13]:
perspective_df["toxic_class"] = (perspective_df[toxic_classes].sum(axis=1) > 0).astype(int)

In [15]:
perspective_df["lemmatize"] = [preprocess_text(text) for text in tqdm_notebook(perspective_df["translate"])]

In [17]:
with open('data/toxicity.pos', "w") as file:
    for comment in perspective_df[perspective_df["toxic_class"] == 1]["lemmatize"]:
        file.write(comment + '\n')

In [18]:
with open('data/toxicity.neg', "w") as file:
    for comment in perspective_df[perspective_df["toxic_class"] == 0]["lemmatize"]:
        file.write(comment + '\n')